**<a id="1"></a>
# <p style="font-family:camtasia;font-size:120%;color:purple;text-align:center;border-radius:15px 50px; padding:7px">Import Libraries </p>

In [ ]:
# Install imbalanced-learn
!pip install imbalanced-learn

# Verify TensorFlow version and install a specific one only if needed
import tensorflow as tf
print("Current TensorFlow version:", tf.__version__)

if tf.__version__ != "2.12.0":
    !pip install tensorflow==2.12


In [ ]:
import os
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import tensorflow as tf
from sklearn.metrics import confusion_matrix, classification_report
from tqdm import tqdm
from imblearn.over_sampling import SMOTE

**<a id="2"></a>
# <p style="font-family:camtasia;font-size:120%;color:purple;text-align:center;border-radius:15px 50px; padding:7px">Read Data </p>

In [ ]:
images = []
labels = []
for subfolder in tqdm(os.listdir('/kaggle/input/alzheimerscnn/Alzheimer_s Dataset/train')):
    subfolder_path = os.path.join('/kaggle/input/alzheimerscnn/Alzheimer_s Dataset/train', subfolder)
    if os.path.isdir(subfolder_path):
        # Loop through image files in the class folder
        for image_filename in os.listdir(subfolder_path):
            image_path = os.path.join(subfolder_path, image_filename)

            # Check if the file is an image (to avoid non-image files)
            if os.path.isfile(image_path):
                images.append(image_path)
                labels.append(subfolder)

# Create a DataFrame with image paths and labels
df = pd.DataFrame({'image': images, 'label': labels})

# Display the first few rows of the DataFrame
df.head()

**<a id="3"></a>
# <p style="font-family:camtasia;font-size:120%;color:purple;text-align:center;border-radius:15px 50px; padding:7px">Visualization of Dataset </p>

In [ ]:
plt.figure(figsize=(15,6))
ax = sns.countplot(x=df.label,palette='Set1')
ax.set_xlabel("Class",fontsize=20)
ax.set_ylabel("Count",fontsize=20)
plt.title('The Number Of Samples For Each Class',fontsize=20)
plt.grid(True)
plt.xticks(rotation=45)
plt.show()

In [ ]:
plt.figure(figsize=(50,50))
for n,i in enumerate(np.random.randint(0,len(df),50)):
    plt.subplot(10,5,n+1)
    img=cv2.imread(df.image[i])
    img=cv2.resize(img,(224,224))
    img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    plt.imshow(img)
    plt.axis('off')
    plt.title(df.label[i],fontsize=25)

<a id="4"></a>
# <p style="font-family:camtasia;font-size:120%;color:purple;text-align:center;border-radius:15px 50px; padding:7px"> Data Augmentation </p>

In [ ]:
Size=(176,176)
work_dr = ImageDataGenerator(
    rescale = 1./255
)
train_data_gen = work_dr.flow_from_dataframe(df,x_col='image',y_col='label', target_size=Size, batch_size=6500, shuffle=False)

In [ ]:
train_data, train_labels = next(train_data_gen)

In [ ]:
class_num=np.sort(['Alzheimer_s disease','Cognitively normal','Early mild cognitive impairment','Late mild cognitive impairment'])
class_num

In [ ]:
sm = SMOTE(random_state=42)
train_data, train_labels = sm.fit_resample(train_data.reshape(-1, 176 * 176 * 3), train_labels)
train_data = train_data.reshape(-1, 176,176, 3)
print(train_data.shape, train_labels.shape)

In [ ]:
labels=[class_num[i] for i in np.argmax(train_labels,axis=1) ]
plt.figure(figsize=(15,6))
ax = sns.countplot(x=labels,palette='Set1')
ax.set_xlabel("Class",fontsize=20)
ax.set_ylabel("Count",fontsize=20)
plt.title('The Number Of Samples For Each Class',fontsize=20)
plt.grid(True)
plt.xticks(rotation=45)
plt.show()

In [ ]:
X_train, X_test1, y_train, y_test1 = train_test_split(train_data,train_labels, test_size=0.3, random_state=42,shuffle=True,stratify=train_labels)
X_val, X_test, y_val, y_test = train_test_split(X_test1,y_test1, test_size=0.5, random_state=42,shuffle=True,stratify=y_test1)
print('X_train shape is ' , X_train.shape)
print('X_test shape is ' , X_test.shape)
print('X_val shape is ' , X_val.shape)
print('y_train shape is ' , y_train.shape)
print('y_test shape is ' , y_test.shape)
print('y_val shape is ' , y_val.shape)

<a id="10"></a>
# <p style="font-family:camtasia;font-size:120%;color:purple;text-align:center;border-radius:15px 50px; padding:7px">CNN Modeling </p>

In [ ]:
model=tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(32,kernel_size=(3,3),strides=2,padding='same',activation='relu',input_shape=(176,176,3)))
model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2),strides=2,padding='same'))
model.add(tf.keras.layers.Conv2D(64,kernel_size=(3,3),strides=2,activation='relu',padding='same'))
model.add(tf.keras.layers.MaxPool2D((2,2),2,padding='same'))
model.add(tf.keras.layers.Conv2D(128,kernel_size=(3,3),strides=2,activation='relu',padding='same'))
model.add(tf.keras.layers.MaxPool2D((2,2),2,padding='same'))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(1024,activation='relu'))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(4,activation='softmax'))
model.summary()

In [ ]:
tf.keras.utils.plot_model(model, to_file='model.png', show_shapes=True, show_layer_names=True,show_dtype=True,dpi=60)

In [ ]:
checkpoint_cb =ModelCheckpoint("CNN_model.keras", save_best_only=True)
early_stopping_cb =EarlyStopping(patience=10, restore_best_weights=True)
model.compile(optimizer ='adam', loss='categorical_crossentropy', metrics=['accuracy'])
hist = model.fit(X_train,y_train, epochs=25, validation_data=(X_val,y_val), callbacks=[checkpoint_cb, early_stopping_cb])

<a id="11"></a>
# <p style="color:purple;text-align:center;border-radius:15px 50px; padding:7px"> Evaluating the Model  </p>

In [ ]:
hist_=pd.DataFrame(hist.history)
hist_

In [ ]:
plt.figure(figsize=(15,8))
plt.subplot(1,2,1)
plt.plot(hist_['loss'],label='Train_Loss')
plt.plot(hist_['val_loss'],label='Validation_Loss')
plt.title('Train_Loss & Validation_Loss',fontsize=20)
plt.legend()
plt.subplot(1,2,2)
plt.plot(hist_['accuracy'],label='Train_Accuracy')
plt.plot(hist_['val_accuracy'],label='Validation_Accuracy')
plt.title('Train_Accuracy & Validation_Accuracy',fontsize=20)
plt.legend()
plt.show()

In [ ]:
score, acc= model.evaluate(X_test,y_test)
print('Test Loss =', score)
print('Test Accuracy =', acc)

In [ ]:
predictions = model.predict(X_test)
y_pred = np.argmax(predictions,axis=1)
y_test_ = np.argmax(y_test,axis=1)
df = pd.DataFrame({'Actual': y_test_, 'Prediction': y_pred})
df

In [ ]:
plt.figure(figsize=(50,50))
for n,i in enumerate(np.random.randint(0,len(X_test),20)):
    plt.subplot(10,2,n+1)
    plt.imshow(X_test[i])
    plt.axis('off')
    plt.title(f'{class_num[y_test_[i]]} ==== {class_num[y_pred[i]]}',fontsize=27)

In [ ]:
CM = confusion_matrix(y_test_,y_pred)
CM_percent = CM.astype('float') / CM.sum(axis=1)[:, np.newaxis]
sns.heatmap(CM_percent,fmt='g',center = True,cbar=False,annot=True,cmap='Blues')
CM

In [ ]:
ClassificationReport = classification_report(y_test_,y_pred)
print('Classification Report is : ', ClassificationReport )